## PSRC Pre-processing
The main purpose of these scripts is to create an merged analytical data set from trip, household, 
and person data sets. The analytical data set is denormalized at the 'trip' level and can be
found at DATA_DIR + 'Trip_Household_Merged.csv'. A simple tableau dashboard built from this set (currently
draft and needs to be upoaded with the revised data) can be accessed at:
https://public.tableau.com/views/PSRC2018/ODPairs?:embed=y&:display_count=yes

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier

from datetime import datetime

DATA_DIR = '../data/'



In [2]:
# Remove first line from .csv, only needed once! Alternatively, load strait from .XLS and ignore the first row.

def removeFirstLine():
    data_dir = '../data/'

    with open(data_dir + '2017-pr2-1-household.csv', 'r') as fin:
        data = fin.read().splitlines(True)
    with open(data_dir + '2017-pr2-1-household.csv', 'w') as fout:
        fout.writelines(data[1:])
    
    with open(data_dir + '2017-pr2-5-trip.csv', 'r') as fin:
        data = fin.read().splitlines(True)
    with open(data_dir + '2017-pr2-5-trip.csv', 'w') as fout:
        fout.writelines(data[1:])
              
    with open(data_dir + '2017-pr2-2-person.csv', 'r') as fin:
        data = fin.read().splitlines(True)
    with open(data_dir + '2017-pr2-2-person.csv', 'w') as fout:
        fout.writelines(data[1:])
              
#removeFirstLine()   

## Process Persons (Race demographics)

In [3]:
# Load race and gender information from the Persons data
df_Person = pd.read_csv(DATA_DIR + '2017-pr2-2-person.csv')
#print(df_Person.dtypes)

df_persons = df_Person[['personid','race_afam','race_aiak','race_asian','race_hapi','race_hisp','race_white','race_other','gender']]
df_persons.dropna(axis = 0, inplace=True)

df_persons['gender_male'] = np.where(df_persons['gender']==1, 1, 0)
df_persons['gender_female'] = np.where(df_persons['gender']==2, 1, 0)
df_persons['gender_another'] = np.where(df_persons['gender']==3, 1, 0)
df_persons['gender_noanswer'] = np.where(df_persons['gender']==4, 1, 0)

print (df_persons.head())


     personid  race_afam  race_aiak  race_asian  race_hapi  race_hisp  \
0  1710000501        0.0        0.0         0.0        0.0        0.0   
1  1710000502        0.0        0.0         0.0        0.0        0.0   
2  1710002401        0.0        0.0         0.0        0.0        0.0   
3  1710002402        0.0        0.0         0.0        0.0        0.0   
5  1710005201        0.0        0.0         0.0        0.0        0.0   

   race_white  race_other  gender  gender_male  gender_female  gender_another  \
0         1.0         0.0       2            0              1               0   
1         1.0         0.0       1            1              0               0   
2         1.0         0.0       3            0              0               1   
3         1.0         0.0       2            0              1               0   
5         1.0         0.0       2            0              1               0   

   gender_noanswer  
0                0  
1                0  
2          

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/py

## Process Households

In [4]:
df_Household = pd.read_csv(DATA_DIR + '2017-pr2-1-household.csv')
df_Blockgroup_UrbanVillage = pd.read_csv(DATA_DIR + 'Blockgroup_UrbanVillage.csv')
#print(df_Person.dtypes)

df_households = df_Household[['final_home_bg','hhid','hhsize','vehicle_count','numchildren',
                              'hhincome_broad','car_share','rent_own','res_dur','offpark','hh_wt_revised']]

df_households['final_home_bg'] = df_households['final_home_bg'].astype(float).astype(int).astype(str)

# merge with seattle block group data
df_seattle = df_Blockgroup_UrbanVillage[['BLOCKGROUP','URBAN_VILLAGE_NAME','URBAN_VILLAGE_TYPE']]
df_seattle['final_home_bg'] = df_seattle['BLOCKGROUP'].astype(str)

df_households = pd.merge(left=df_households, right=df_seattle, how='left', on='final_home_bg')
df_households.drop(['BLOCKGROUP'], axis = 1, inplace=True)
df_households['URBAN_VILLAGE_NAME'] = df_households['URBAN_VILLAGE_NAME'].fillna("Outside Seattle")
df_households['URBAN_VILLAGE_TYPE'] = df_households['URBAN_VILLAGE_TYPE'].fillna("Outside Seattle")

df_households = df_households.rename(columns={'final_lat':'hh_lat', 'final_lng':'hh_lng'})
df_households = df_households.rename(columns={'URBAN_VILLAGE_NAME':'hh_uv', 'URBAN_VILLAGE_TYPE':'hh_uv_type'})

# Assign income variables
df_households['income'] = np.where(df_households['hhincome_broad']==1, "Under $25,000", "")
df_households['income'] = np.where(df_households['hhincome_broad']==2, "$25,000-$49,999", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==3, "$50,000-$74,999", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==4, "$75,000-$99,999", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==5, "$100,000 or more", df_households['income'])
df_households['income'] = np.where(df_households['hhincome_broad']==6, "Prefer not to answer", df_households['income'])

# Assign home ownership
df_households['homeownership'] = np.where(df_households['rent_own']==1, "Own", "Other")
df_households['homeownership'] = np.where(df_households['rent_own']==2, "Rent", df_households['homeownership'])

# Assign residency tenure
df_households['tenure'] = np.where(df_households['res_dur']==1, "Less than a year", "")
df_households['tenure'] = np.where(df_households['res_dur']==2, "Between 1 and 2 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==3, "Between 2 and 3 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==4, "Between 3 and 5 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==5, "Between 5 and 10 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==6, "Between 10 and 20 years", df_households['tenure'])
df_households['tenure'] = np.where(df_households['res_dur']==7, "More than 20 years", df_households['tenure'])

#df_households = pd.merge(left=df_households, right=df_race, how='left', on='hhid')
#df_households = pd.merge(left=df_households, right=df_gender, how='left', on='hhid')

print (df_households.head())


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


  final_home_bg      hhid  hhsize  vehicle_count  numchildren  hhincome_broad  \
0  530330323234  17100005       2              2            0               4   
1  530330076002  17100024       3              1            1               4   
2  530330075005  17100052       1              0            0               1   
3  530330084001  17100059       1              0            0               5   
4  530530720002  17100060       1              1            0               1   

   car_share  rent_own  res_dur  offpark  hh_wt_revised            hh_uv  \
0          2         1        7        2      24.441709  Outside Seattle   
1          1         1        2        1      26.224981   Madison-Miller   
2          2         2        6        1      25.692826     Capitol Hill   
3          2         2        1        0      47.768728        Pike/Pine   
4          2         2        5        2     278.147224  Outside Seattle   

                  hh_uv_type            income homeowner

## Process Trips

In [5]:
df_Trip = pd.read_csv(DATA_DIR + '2017-pr2-5-trip.csv')
print (list(df_Trip.columns.values))

df_trips = df_Trip[['tripid','hhid', 'depart_time_hhmm','arrival_time_hhmm',
                    'o_bg','d_bg','personid','google_duration','trip_path_distance',
                    'daynum','origin_purpose','dest_purpose',
                    'mode_1','travelers_total','traveldate','trip_weight_revised']]

#print(df_trips.dtypes)

# drop null blockgroups
df_trips = df_trips.dropna(subset=['o_bg', 'd_bg'])

df_trips['d_bg'] = df_trips['d_bg'].astype(int).astype(str)
df_trips['o_bg'] = df_trips['o_bg'].astype(int).astype(str)

# merge with seattle block group data
df_seattle = df_Blockgroup_UrbanVillage[['BLOCKGROUP','URBAN_VILLAGE_NAME','URBAN_VILLAGE_TYPE']]
df_seattle['BLOCKGROUP'] = df_seattle['BLOCKGROUP'].astype(str)
                              
df_trips = pd.merge(left=df_trips, right=df_seattle, how='left', left_on='o_bg', right_on='BLOCKGROUP')
df_trips = df_trips.rename(columns={'URBAN_VILLAGE_NAME':'uv_origin', 'URBAN_VILLAGE_TYPE':'uvType_origin'})
df_trips.drop(['BLOCKGROUP'], axis = 1, inplace=True)

df_trips['uv_origin'] = df_trips['uv_origin'].fillna("Outside Seattle")
df_trips['uvType_origin'] = df_trips['uvType_origin'].fillna("Outside Seattle")

df_trips = pd.merge(left=df_trips, right=df_seattle, how='left', left_on='d_bg', right_on='BLOCKGROUP')
df_trips = df_trips.rename(columns={'URBAN_VILLAGE_NAME':'uv_dest', 'URBAN_VILLAGE_TYPE':'uvType_dest'})
df_trips.drop(['BLOCKGROUP'], axis = 1, inplace=True)

df_trips['uv_dest'] = df_trips['uv_dest'].fillna("Outside Seattle")
df_trips['uvType_dest'] = df_trips['uvType_dest'].fillna("Outside Seattle")

# Drop missing variables, clean up column
df_trips['mode_1'] = df_trips['mode_1'].fillna(0)
df_trips['mode_1'] = df_trips['mode_1'].astype(str).replace(' ', '0')
df_trips['mode_1'] = df_trips['mode_1'].astype(str).astype(int)
df_trips['travelers_total'] = df_trips['travelers_total'].astype(str).replace(' ', '0')
df_trips['travelers_total'] = df_trips['travelers_total'].astype(str).astype(int)

# drop rows where the duration or distance is null or an empty space
df_trips = df_trips[df_trips['google_duration'].notnull()]
df_trips = df_trips[df_trips['trip_path_distance'].notnull()]
df_trips = df_trips[df_trips['google_duration'] != " "]
df_trips = df_trips[df_trips['trip_path_distance'] != " "]
df_trips['google_duration'] = df_trips['google_duration'].astype(float)
df_trips['trip_path_distance'] = df_trips['trip_path_distance'].astype(float)
df_trips = df_trips[df_trips['trip_path_distance'].notnull()]

# Create OD Pairs for urban villages and block groups
df_trips['uv_od_pair'] = df_trips['uv_origin'] + " to " + df_trips['uv_dest']
df_trips['bg_od_pair'] = df_trips['o_bg'].astype(str) + " to " + df_trips['d_bg'].astype(str)

# Assign mode variables
df_trips['mode'] = np.where(df_trips['mode_1']==1, "Walk", "Other")
df_trips['mode'] = np.where(df_trips['mode_1']==2, "Bike", df_trips['mode'])

df_trips['mode'] = np.where((df_trips['mode_1']>=3) & (df_trips['mode_1']<=17)  & (df_trips['travelers_total']==1), "Drive Alone", df_trips['mode'])
df_trips['mode'] = np.where(((df_trips['mode_1']==21) | (df_trips['mode_1']==22) |
                                 (df_trips['mode_1']==33) | (df_trips['mode_1']==34) | (df_trips['mode_1']==18))
                                 & (df_trips['travelers_total']==1), "Drive Alone", df_trips['mode'])

df_trips['mode'] = np.where((df_trips['mode_1']>=3) & (df_trips['mode_1']<=17)  & (df_trips['travelers_total']!=1), "Drive w Others", df_trips['mode'])
df_trips['mode'] = np.where(((df_trips['mode_1']==21) | (df_trips['mode_1']==22) |
                                 (df_trips['mode_1']==33) | (df_trips['mode_1']==34) | (df_trips['mode_1']==18))
                                 & (df_trips['travelers_total']>1), "Drive w Others", df_trips['mode'])

df_trips['mode'] = np.where((df_trips['mode_1']==23) | (df_trips['mode_1']==41) | (df_trips['mode_1']==42), "Transit", df_trips['mode'])
df_trips['mode'] = np.where((df_trips['mode_1']==32) | (df_trips['travelers_total']==52), "Transit", df_trips['mode'])

df_trips['drive_alone'] = np.where(df_trips['mode']=="Drive Alone", 1, 0)


# Assign purpose variables
df_trips['purpose'] = np.where(df_trips['dest_purpose']==1, "Go Home", "Other")
df_trips['purpose'] = np.where(df_trips['dest_purpose']==6, "School", df_trips['purpose'])
df_trips['purpose'] = np.where((df_trips['dest_purpose']==10) | (df_trips['dest_purpose']==11), "Work", df_trips['purpose'])

# Assign time period variables
df_trips['depart_time'] = pd.to_datetime(df_trips['depart_time_hhmm'], errors='coerce')
df_trips['depart_day'] = pd.to_datetime(df_trips['traveldate'], errors='coerce')

df_trips['depart_day'] = df_trips['depart_day'].dt.dayofweek
df_trips['depart_time'] = df_trips['depart_time'].dt.hour

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>=7) & (df_trips['depart_time']<=9), "Weekday AM", "")

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>=10) & (df_trips['depart_time']<=15), "Weekday Mid", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>=16) & (df_trips['depart_time']<=19), "Weekday PM", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']<7), "Other", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=0) & (df_trips['depart_day']<=4) & 
                                     (df_trips['depart_time']>19), "Late Night", df_trips['depart_period'])

df_trips['depart_period'] = np.where((df_trips['depart_day']>=5) & (df_trips['depart_day']<=6), "Late Night", df_trips['depart_period'])

#dts = dfBad[cols].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%m/%d/%Y'))
#pd.to_datetime(df_trips['depart_time_timestamp'])

#df_trips['depart_time_timestamp'] = datetime.strptime(df_trips['depart_time_timestamp'], '%b %d %Y %I:%M%p')
#to_datetime

#print (df_trips.dtypes)
print (df_trips.head())

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,18,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,68,69,80,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['recid', 'hhid', 'personid', 'pernum', 'tripid', 'tripnum', 'tottrip1', 'tottrip2', 'daynum', 'hhgroup', 'origin_home', 'dest_home', 'depart_time_mam', 'depart_time_hhmm', 'arrival_time_mam', 'arrival_time_hhmm', 'reported_duration', 'google_duration', 'trip_path_distance', 'speed_mph', 'activity_duration', 'origin_purpose', 'o_purpose_other', 'dest_purpose', 'dest_purpose_other', 'mode_1', 'mode_2', 'mode_3', 'mode_4', 'travelers_hh', 'travelers_nonhh', 'travelers_total', 'driver', 'hhmember1', 'hhmember2', 'hhmember3', 'hhmember4', 'hhmember5', 'hhmember6', 'hhmember7', 'hhmember8', 'hhmember9', 'hhmember_none', 'pool_start', 'change_vehicles', 'park_ride_area_start', 'park_ride_area_end', 'park_ride_lot_start', 'park_ride_lot_end', 'toll', 'toll_pay', 'taxi_type', 'taxi_pay', 'bus_type', 'bus_pay', 'bus_cost_dk', 'ferry_type', 'ferry_pay', 'ferry_cost_dk', 'rail_type', 'rail_pay', 'rail_cost_dk', 'air_type', 'air_pay', 'airfare_cost_dk', 'mode_acc', 'mode_egr', 'park', 'park_type',

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


          tripid      hhid depart_time_hhmm arrival_time_hhmm          o_bg  \
0  1710000501001  17100005          9:55 AM          10:05 AM  530330323234   
1  1710000501002  17100005         10:10 AM          10:15 AM  530330323132   
2  1710000501003  17100005         10:50 AM          11:00 AM  530330323133   
3  1710000501004  17100005         11:35 AM          12:00 PM  530330323234   
4  1710000501005  17100005          3:10 PM           3:30 PM  530330232022   

           d_bg    personid  google_duration  trip_path_distance  daynum  \
0  530330323132  1710000501              7.0                2.30       1   
1  530330323133  1710000501              4.0                1.12       1   
2  530330323234  1710000501              7.0                3.26       1   
3  530330232022  1710000501             18.0                8.13       1   
4  530330323234  1710000501             16.0                8.04       1   

       ...               uv_dest      uvType_dest  \
0      ...     

## Merged, Normalized Trip Dataset
Add household and person information to each trip

In [7]:
df_trip_household= pd.merge(left=df_trips, right=df_households, how='left', left_on='hhid', right_on='hhid')
df_trip_household= pd.merge(left=df_trip_household, right=df_persons, how='left', left_on='personid', right_on='personid')

print (df_trip_household.head())

df_trip_household.to_csv(DATA_DIR + 'Trip_Household_Merged.csv', mode='w', header=True, index=False)
#df_UV_Origins.to_csv(data_dir + 'UV_Origins.csv', mode='w', header=True, index=False)

          tripid      hhid depart_time_hhmm arrival_time_hhmm          o_bg  \
0  1710000501001  17100005          9:55 AM          10:05 AM  530330323234   
1  1710000501002  17100005         10:10 AM          10:15 AM  530330323132   
2  1710000501003  17100005         10:50 AM          11:00 AM  530330323133   
3  1710000501004  17100005         11:35 AM          12:00 PM  530330323234   
4  1710000501005  17100005          3:10 PM           3:30 PM  530330232022   

           d_bg    personid  google_duration  trip_path_distance  daynum  \
0  530330323132  1710000501              7.0                2.30       1   
1  530330323133  1710000501              4.0                1.12       1   
2  530330323234  1710000501              7.0                3.26       1   
3  530330232022  1710000501             18.0                8.13       1   
4  530330323234  1710000501             16.0                8.04       1   

        ...        race_asian  race_hapi  race_hisp  race_white race

## Blockgroup Pairs
This section calculates all combinations of blockgroups pairs.

In [ ]:
# load King County blockgroup file that has centroids.

df_Blockgroups = pd.read_csv(data_dir + 'KingCountyCentroids.csv') 
df_Blockgroups['geoid'] = df_Blockgroups['geoid'].astype(str)

df_Merged= pd.merge(left=df_trips, right=df_Blockgroups, how='inner', left_on='d_bg', right_on='geoid')
#print (list(df_Merged.columns.values))

df_Merged['bg_pair'] = df_Merged['o_bg'].astype(str) + "-" + df_Merged['d_bg'].astype(str)

df_Merged_agg = df_Merged.groupby(['bg_pair'], as_index=False).agg({'o_bg':['max'],'d_bg':['max'],
                                                                   'centroid lat':['max'],'centroid long':['max']})
df_Merged_agg.columns = df_Merged_agg.columns.droplevel(level=1)
df_Merged_agg = df_Merged_agg.rename(index=str, columns={"centroid lat": "lat_dest", "centroid long": "lon_dest"})

df_Merged_agg= pd.merge(left=df_Merged_agg, right=df_Blockgroups, how='inner', left_on='o_bg', right_on='geoid')
df_Merged_agg = df_Merged_agg.rename(index=str, columns={"centroid lat": "lat_origin", "centroid long": "lon_origin"})

df_Merged_agg.to_csv(data_dir + 'blockgroup_pairs_kc.csv', mode='w', header=True, index=False)

print (df_Merged_agg)